# Entrenamiento usando como base un modelo VGG16.

## Inicialización del código.

Librerías necesarias para trabajar.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import math
import csv
import time
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import matplotlib.cm as cm
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, Concatenate, MaxPooling2D, UpSampling2D, Cropping2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, RepeatVector, Reshape
from tensorflow.keras.metrics import binary_accuracy, Recall, BinaryCrossentropy
from tensorflow.keras.applications import VGG16

Constantes.

In [ ]:
NUM_EPOCHS = 10
BATCH_SIZE = 32
THRESHOLD = 0.5
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.25
RANDOM_STATE = None
NUM_WORKERS = 1
USE_MULTIPROCESSING = False
WIDTH_IMAGES = 256
HEIGHT_IMAGES = 256
SAVE_MODEL = True

In [ ]:
DIR_CSV = 'CXR8'
DIR_IMAGES = r'CXR8\images'
DIR_RESULTS = 'Results'
CSV_TRAINING = 'Data_Entry_2017_v2020.csv'
COLUMN_IMAGE = 'Image Index'
COLUMN_LABELS = 'Finding Labels'
COLUMN_FEATURE_AGE = 'Patient Age'
COLUMN_FEATURE_GENDER = 'Patient Gender'
COLUMN_FEATURE_VIEW_POSITION = 'View Position'
ADDITIONAL_DATA = [COLUMN_FEATURE_AGE, COLUMN_FEATURE_GENDER, COLUMN_FEATURE_VIEW_POSITION]
LABEL_NO_FINDING = 'No Finding'
date_test = datetime.now()
actual_date = date_test.strftime('%Y-%m-%d.%H-%M-%S')

Prepara el entorno de colab.

In [ ]:
DIR_GOOGLE_DRIVE = '/content/drive/'

from google.colab import drive
drive.mount(DIR_GOOGLE_DRIVE, force_remount=True)
CRX8_DOWNLOAD_FILES = 1

Asegurate que existen los directorios necesarios para guardar los resultados.

In [ ]:
if not os.path.exists(DIR_IMAGES):
    os.makedirs(DIR_IMAGES)
if not os.path.exists(os.path.join(DIR_RESULTS, 'images')):
    os.makedirs(os.path.join(DIR_RESULTS, 'images'))

## Carga y preparación de los datos.

### Descarga de los conjuntos de imágenes. 

Descarga y descomprime los archivos de imágenes desde la web de ChestX-ray14 (https://nihcc.app.box.com/v/ChestXray-NIHCC).

In [ ]:
%%time
%cd /content/
links = [
    'https://nihcc.box.com/shared/static/vfk49d74nhbxq3nqjg0900w5nvkorp5c.gz',	# image_01.tar.gz
    'https://nihcc.box.com/shared/static/i28rlmbvmfjbl8p2n3ril0pptcmcu9d1.gz',	# image_02.tar.gz
    'https://nihcc.box.com/shared/static/f1t00wrtdk94satdfb9olcolqx20z2jp.gz',	# image_03.tar.gz
    'https://nihcc.box.com/shared/static/0aowwzs5lhjrceb3qp67ahp0rd1l1etg.gz',	# image_04.tar.gz
    'https://nihcc.box.com/shared/static/v5e3goj22zr6h8tzualxfsqlqaygfbsn.gz',	# image_05.tar.gz
    'https://nihcc.box.com/shared/static/asi7ikud9jwnkrnkj99jnpfkjdes7l6l.gz',	# image_06.tar.gz
    'https://nihcc.box.com/shared/static/jn1b4mw4n6lnh74ovmcjb8y48h8xj07n.gz',	# image_07.tar.gz
    'https://nihcc.box.com/shared/static/tvpxmn7qyrgl0w8wfh9kqfjskv6nmm1j.gz',	# image_08.tar.gz
    'https://nihcc.box.com/shared/static/upyy3ml7qdumlgk2rfcvlb9k6gvqq2pj.gz',	# image_09.tar.gz
    'https://nihcc.box.com/shared/static/l6nilvfa9cg3s28tqv1qc1olm3gnz54p.gz',	# image_10.tar.gz
    'https://nihcc.box.com/shared/static/hhq8fkdgvcari67vfhs7ppg2w6ni4jze.gz',	# image_11.tar.gz
    'https://nihcc.box.com/shared/static/ioqwiy20ihqwyr8pf4c24eazhh281pbu.gz'	# image_12.tar.gz
]

for n, gz_file_url in enumerate(links):
    if n == CRX8_DOWNLOAD_FILES:
        break;
    gz_file_name = 'images_%02d.tar.gz' % (n + 1)
    !wget {gz_file_url} -O {gz_file_name}
    !tar -xzf {gz_file_name}
    !rm {gz_file_name}

### Carga de los datos.

Carga el dataset con las etiquetas y datos de cada radiografía y lo muestra.

In [ ]:
df_xrays = pd.read_csv(os.path.join(DIR_CSV, CSV_TRAINING))
df_xrays

Obtiene las distintas etiquetas.

In [ ]:
#  Recupera los valores de las clases, pero sin repetirse y ordenados alfabéticamente.
labels = list(set(label for labels in df_xrays[COLUMN_LABELS].unique() for label in labels.split('|')))

labels = np.sort(labels)

#  La etiqueta que representa que no hay enfermedad la vamos a tratar de forma diferente.
# En realidad esto sería que no hay ninguna etiqueta de enfermedad. No tendría sentido que
# se pudiera dar, por ejemplo, 'no finding|pneumonia'. Por lo tanto hay que eliminarla.
# La forma que menos problemas ha dado, es al principio dejarla como una etiqueta más.
# Una vez binarizadas las etiquetas, eliminar la columna.
labels = np.concatenate(((np.delete(labels, np.where(labels == LABEL_NO_FINDING)[0]), [LABEL_NO_FINDING])))

#  Cuenta las clases. Restamos la del 'No finding'.
num_labels = len(labels) - 1

### Arreglo de los datos.

Código para pruebas, cuando no se trabaja con todo el conjunto de imágenes.

Mira los archivos que hay en el directorio de imágenes y selecciona sólo las filas correctas del dataframe.

In [ ]:
list_xrays = os.listdir(DIR_IMAGES)
df_xrays = df_xrays[df_xrays[COLUMN_IMAGE].isin(list_xrays)].copy()
del list_xrays

Agrega la ruta del directorio de las imágenes al nombre de cada una.

In [ ]:
df_xrays[COLUMN_IMAGE] = df_xrays[COLUMN_IMAGE].apply(lambda i: os.path.join(DIR_IMAGES, i))

Binariza las etiquetas en el dataframe.

In [ ]:
#  Crear una instancia de MultiLabelBinarizer.
mlb = MultiLabelBinarizer(classes=labels)

#  Transforma las clases del dataframe.
vectors_one_hot_from_labels = list(mlb.fit_transform([txt_labels.split('|') for txt_labels in df_xrays[COLUMN_LABELS]]))

#  Elimina la columna correspondiente al 'No finding'.
vectors_one_hot_from_labels = np.delete(vectors_one_hot_from_labels, -1, axis=1)

#  Elimina ahora la etiqueta del 'No finding' de labels.
labels = np.delete(labels, np.where(labels == LABEL_NO_FINDING)[0])

#  Actualiza el dataframe.
df_xrays[COLUMN_LABELS] = list(vectors_one_hot_from_labels)

del vectors_one_hot_from_labels
del mlb

Codifica los datos adicionales

In [ ]:
#  Codificamos la columna de género, M por 0 y V por 1.
df_xrays[COLUMN_FEATURE_GENDER] = df_xrays[COLUMN_FEATURE_GENDER].map({'M': 0, 'F': 1})

#  Normalizamos la edad. Supondremos que las edades posibles estén comprendidas entre 0 
# y 100.
df_xrays[COLUMN_FEATURE_AGE] /= 100.

#  Codificamos la columna con la posición de la imagen, PA (Posteroanterior) por 0 y 
# AP (anteroposterior) por 1.
df_xrays[COLUMN_FEATURE_VIEW_POSITION] = df_xrays[COLUMN_FEATURE_VIEW_POSITION].map({'PA': 0, 'AP': 1})

Muestra el conjunto resultante.

In [ ]:
df_xrays

## Entrenamiento.

Calcula los pesos de cada etiqueta para reforzar el aprendizaje de casos positivos.

In [ ]:
#  Obten las proporciones de muestras positivas por etiqueta.
positives_ratio = np.sum(df_xrays[COLUMN_LABELS].to_list(), axis=0) / len(df_xrays)

# Crea el diccionario de pesos.
label_weights = {label: 1/ratio for label, ratio in enumerate(positives_ratio)}

Prepara los conjuntos de entrenamiento, validación y prueba.

In [ ]:
#  Divide todo el conjunto en entrenamiento y prueba.
df_xrays_train, df_xrays_test = train_test_split(df_xrays, test_size=TEST_SIZE, random_state=RANDOM_STATE)
del df_xrays

# Divide el conjunto de entrenamiento recien creado en entrenamiento y validación.
df_xrays_train, df_xrays_val = train_test_split(df_xrays_train, test_size=TEST_SIZE, random_state=RANDOM_STATE)

Crea la red utilizando el modelo preentrenado VGG16.

In [ ]:
#  Parte densa para los datos adicionales.
def dense_branch(inputs):
    fully_conn = Dense(64, activation=tf.nn.relu, name='additional_info_dense_1')(inputs)
    fully_conn = Dense(32, activation=tf.nn.relu, name='additional_info_dense_2')(fully_conn)
    
    return fully_conn

#  Carga el modelo VGG16 preentrenado sin las capas densas incluidas.
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(WIDTH_IMAGES, HEIGHT_IMAGES, 3))

#  Marca las capas del modelo preentrenado como no entrenables.
for layer in base_model.layers:
    layer.trainable = False

#  Agregar capas densas adicionales
pretrain_model = base_model.output
pretrain_model = GlobalAveragePooling2D()(pretrain_model)


#  La entrada serán los datos adicionales de la imagen, por lo tanto se corresponde con el ancho y el alto. Al ser
# en blanco y negro sólo es necesario un canal.
inputs_dense_block = Input(shape=(len(ADDITIONAL_DATA),), name='additional_info')
dense_block = dense_branch(inputs_dense_block)

#  Fusiona cada rama
fusion_layer = Concatenate()([pretrain_model, dense_block])

#  Pademos el resultado a las capas densas.
fully_conn = Dense(128, activation=tf.nn.relu, name='dense_1')(fusion_layer)
fully_conn = Dense(64, activation=tf.nn.relu, name='dense_2')(fully_conn)
fully_conn = Dense(32, activation=tf.nn.relu, name='dense_3')(fully_conn)
outputs = Dense(num_labels, activation=tf.nn.sigmoid, name='outputs')(fully_conn)

model = Model(inputs=[base_model.input, inputs_dense_block], outputs=outputs)

Compila el modelo.

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[binary_accuracy]
)

Muestra el diagrama del modelo.

In [ ]:
tf.keras.utils.plot_model(model, to_file=os.path.join(DIR_RESULTS, 'images', 'model-{}.png'.format(actual_date)), show_shapes=True)

Muestra el resumen de la arquitectura.

In [ ]:
summary_report = model.summary()
summary_report

Generador personalizado de datos, para ir cargando los datos por lotes al entrenar.

In [ ]:
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size, img_width, img_height, labels, shuffle=True):
        self.df = df.copy()
        self.batch_size = batch_size
        self.img_width = img_width
        self.img_height = img_height
        self.additional_info_cols = [COLUMN_FEATURE_AGE, COLUMN_FEATURE_GENDER]
        self.labels = labels
        self.num_classes = len(labels)
        self.shuffle = shuffle
        if shuffle:
            self.on_epoch_end()
        
    def __len__(self):
        # Número de lotes que devolverá este generador.
        return int(np.ceil(len(self.df) / self.batch_size))
    
    def __getitem__(self, index):
        #  Devuelve un lote.
        #  Selecciona las filas del dataframe correspondientes al lote index.
        batch_df = self.df[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(batch_df)
        return X, y
    
    def __data_generation(self, batch_df):
        batch_images = []
        batch_labels = []
        batch_additional_info = []

        for i, row in batch_df.iterrows():
            path_img = row[COLUMN_IMAGE]
            tensor_img = tf.io.read_file(path_img)
            tensor_img = tf.image.decode_png(tensor_img)
            
            #  Redimensiona la imagen.
            tensor_img = tf.image.resize(tensor_img, [self.img_width, self.img_height])

            #  Algunas imágenes tienen 4 canales. Hay que pasarles a grises.
            if tf.shape(tensor_img)[2] > 1:
                #  Elimina el canal alpha.
                tensor_img = tf.slice(tensor_img, [0, 0, 0], [self.img_width, self.img_height, 3])

                #  Pasa a escala de grises.
                tensor_img = tf.image.rgb_to_grayscale(tensor_img)

            #  Normaliza los valores de los píxeles a un rango de [-1, 1].
            tensor_img = (tf.cast(tensor_img, tf.float32) - 127.5) / 127.5

            #  Aquí estamos trabajando con un modelo preentrenado que requiere una entrada
            # de 3 canales.
            tensor_img = np.concatenate((tensor_img, tensor_img, tensor_img), axis=2)

            batch_images.append(img_to_array(tensor_img))
            
            #  Agregra las variables con la información adicional.
            additional_info = row[ADDITIONAL_DATA].values.astype(np.float32)
            batch_additional_info.append(additional_info)
            
            #  Agrega las etiquetas de cada radiografía.
            batch_labels.append(row[COLUMN_LABELS])
        
        X = [
             np.array(batch_images),
             np.asarray(batch_additional_info).astype('float32')
        ]
        y = np.array(batch_labels)
        
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1)

Entrena y evalua el modelo.

In [ ]:
train_generator = CustomDataGenerator(df_xrays_train, batch_size=BATCH_SIZE,
                                      img_width=WIDTH_IMAGES, img_height=HEIGHT_IMAGES, 
                                      labels=labels, shuffle=True
)

validation_generator = CustomDataGenerator(df_xrays_val, batch_size=BATCH_SIZE,
                                      img_width=WIDTH_IMAGES, img_height=HEIGHT_IMAGES, 
                                      labels=labels, shuffle=True
)

start_time = time.time()
history = model.fit(train_generator, 
                    epochs=NUM_EPOCHS,
                    validation_data=validation_generator,
                    steps_per_epoch=math.ceil(df_xrays_train.shape[0] / BATCH_SIZE),
                    shuffle=True,
                    workers=NUM_WORKERS,
                    use_multiprocessing=USE_MULTIPROCESSING,
                    class_weight=label_weights
)
training_time = time.time() - start_time

Muestra las gráficas de perdida y precisión binaria por ciclo.

In [ ]:
# Crea la figura y las subplots
fig, (ax1, ax2) = plt.subplots(1, 2,  figsize=(15, 5))

# Gráfica de perdida por ciclo.
ax1.set_title('Loss / Epoch')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.plot(history.history['loss'])

# Gráfica de precisión binaria por ciclo.
ax2.set_title('Binary Accuracy / Epoch')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Binary Accuracy')
ax2.plot(history.history['binary_accuracy'])

plt.savefig(os.path.join(DIR_RESULTS, 'images', 'loss-binary-accuracy-{}.png'.format(actual_date)), bbox_inches='tight')
plt.show()

## Evaluación.

Realiza la evaluación del modelo.

In [ ]:
test_generator = CustomDataGenerator(df_xrays_test, batch_size=BATCH_SIZE,
                                      img_width=WIDTH_IMAGES, img_height=HEIGHT_IMAGES, 
                                      labels=labels, shuffle=False
)
labels_predictions_scores = model.predict(test_generator)

#  Pon a 1 aquellas que sean mayores o iguales al umbral y 0 al resto.
labels_predictions = np.where(labels_predictions_scores > THRESHOLD, 1, 0)

#  Prepara también un array con las etiquetas auténticas de cada caso de test.
labels_test = np.concatenate(df_xrays_test[COLUMN_LABELS].to_numpy()).reshape(labels_predictions.shape)

Guarda los resultados en csv, por si hay que revisarlos a mano.

In [ ]:
with open(os.path.join(DIR_RESULTS, 'Tests.{}.csv'.format(actual_date)), 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    for row in labels_test:
        writer.writerow(row)

with open(os.path.join(DIR_RESULTS, 'Predictions.{}.csv'.format(actual_date)), 'w', newline='') as f:
    writer = csv.writer(f, delimiter='\t')
    for row in labels_predictions:
        writer.writerow(row)

Matriz de confusión.

In [ ]:
ml_confusion_matrix = multilabel_confusion_matrix(labels_test, labels_predictions)
print(ml_confusion_matrix)

Métricas del módelo.

In [ ]:
cl_report = classification_report(labels_test, labels_predictions, target_names=labels, zero_division=0)
print(cl_report)

Curva ROC de cada etiqueta.

In [ ]:
fpr = dict()
tpr = dict()
thresholds = dict()
roc_auc = dict()
for i in range(len(labels)):
    fpr[i], tpr[i], thresholds[i] = roc_curve(labels_test[:, i], labels_predictions_scores[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

fpr_macro, tpr_macro, _ = roc_curve(labels_test.ravel(), labels_predictions_scores.ravel())
roc_auc_macro = auc(fpr_macro, tpr_macro)

# Dibujar gráficas ROC
cols = 4
rows = math.ceil((len(labels) + 1) / cols)
fig, ax = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4))
for i, label in enumerate(labels):
    row = i // cols
    col = i % cols
    ax[row][col].plot([0, 1], [0, 1], linestyle='--', lw=2, color='red', alpha=0.8)
    ax[row][col].plot(fpr[i], tpr[i], color='blue', lw=2, alpha=0.8) 
    ax[row][col].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title='Curva ROC {0} (AUC = {1:0.2f})'.format(labels[i], roc_auc[i]))
i = i + 1
row = i // cols
col = i % cols
ax[row][col].plot([0, 1], [0, 1], linestyle='--', lw=2, color='red', alpha=0.8)
ax[row][col].plot(fpr_macro, tpr_macro, color='blue', lw=2, alpha=0.8)
ax[row][col].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05], title='Curva ROC macro promediada (AUC = {0:0.2f})'.format(roc_auc_macro))
for i in range(col + 1, cols):
    ax[row][i].axis('off')
plt.tight_layout()

plt.savefig(os.path.join(DIR_RESULTS, 'images', 'roc-curves-{}.png'.format(actual_date)), bbox_inches='tight')
plt.show()

Representa visualmente imágenes que ha clasificado bien y mal.

In [ ]:
num_rows = 40
num_cols = 3

# matplotlib.use('ps')
# rc('text', usetex = True)
# rc('text.latex', preamble = 'usepackage{color}')
fig, ax = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(6 * num_cols, 3 * num_rows), dpi=128, facecolor='none')
plt.subplots_adjust(wspace=2.5, hspace=0.01)

def draw_images():
    test = row = col = 0
    for i in range(len(test_generator)):
        X_test, y_test = test_generator[i]
        X_test = X_test[0]
        for X, y in zip(X_test, y_test):
            list_labels_true = []
            list_labels_pred = []

            #  Comprueba que enfermedades tienen en común el test y la predicción.
            for n, label in enumerate(labels):
                if y[n] == labels_predictions[test][n] and y[n] == 1:
                    #  La radiografía del test tenía esta enfermedad y se ha clasificado bien.            
                    list_labels_true.append(label)
                    list_labels_pred.append(label + ' [' +
                                    '{:.2f}'.format(labels_predictions_scores[test][n] * 100.) +
                                    '%]')
                elif y[n] != labels_predictions[test][n] and y[n] == 1:
                    #  La radiografía del test tenía esta enfermedad, pero no se ha clasificado bien.
                    list_labels_true.append(label)
                elif y[n] != labels_predictions[test][n] and y[n] == 0:
                    #  La radiografía del test no tenía esta enfermedad, y se ha clasificado mal.
                    list_labels_pred.append(label + ' [' +
                                    '{:.2f}'.format(labels_predictions_scores[test][n] * 100.) +
                                    '%]')
                    
            #  Comprueba los casos del 'No Finding' y los colores de los marcos.
            if len(list_labels_true) == 0 and len(list_labels_pred) == 0:
                #  Si la radiografía del test no tenía ninguna enfermedad y no se ha clasificado
                # ninguna, pon ambos textos en verde. Pon el marco de la imagen en verde también.
                text_labels_true = LABEL_NO_FINDING
                text_labels_pred = LABEL_NO_FINDING
                box_img_color = 'green'
            elif len(list_labels_true) == 0:
                #  Si la radiografía no tenía ninguna enferemedad, pero se ha clasificado alguna,
                # pon el texto en rojo (la predicción ha sido totalmente incorrecta).
                text_labels_true = LABEL_NO_FINDING
                box_img_color = 'red'
            elif len(list_labels_pred) == 0:
                #  Si no se ha clasificado ninguna enfermedad pero la radiografía tenía alguna,
                # pon el texto de la predicción y el marco de la imagen en rojo.
                text_labels_pred = LABEL_NO_FINDING
                box_img_color = 'red'
            else:
                #  La radiografía tiene alguna enfermedad y se han clasificado también algunas.
                if np.array_equal(y, labels_predictions[test]):
                    #  Los vectores son iguales, eso es que la clasificación ha sido perfecta.
                    # Ponemos el marco en verde
                    box_img_color = 'green'
                elif np.count_nonzero(y & labels_predictions[test]) > 0:
                    #  Hay alguna enfermedad que se ha clasificado bien. Ponemos el marco en 
                    # amarillo
                    box_img_color = 'yellow'
                else:
                    #  No ha clasificado bien ninguna enferemedad. Ponemos el marco en rojo.
                    box_img_color = 'red'

            #  Formamos los textos en los casos que correspondan.
            if len(list_labels_true) > 0:
                text_labels_true = ', '.join(list_labels_true)

            if len(list_labels_pred) > 0:
                text_labels_pred = ', '.join(list_labels_pred)

            #  Quita los ejes y la rejilla.
            ax[row][col].axis('off')
            ax[row][col].grid(False)
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            ax[row][col].set_xticklabels([])
            ax[row][col].set_yticklabels([])

            #  Dibuja la imagen.
            ax[row][col].imshow(((X * 127) + 128).astype(np.uint8), extent=[0, 128, 0, 128], cmap='gray')
            
            #  Agrega el marco de borde 5
            ax[row][col].add_artist(plt.Rectangle((0, 0),
                            WIDTH_IMAGES - 1, HEIGHT_IMAGES - 1,
                            fill=False,
                            edgecolor=box_img_color,
                            lw = 10)
            )
        
            #  Pon las enfermedades verdaderas.
            ax[row][col].text(0, -0.1,
                              f'True: {text_labels_true}',
                              fontsize=14,
                              color=box_img_color,
                              ha='left',
                              transform=ax[row][col].transAxes
            )

            #  Pon las enfermedades clasificadas.
            ax[row][col].text(0, -0.2,
                              f'Pred: {text_labels_pred}',
                              fontsize=14,
                              color=box_img_color,
                              ha='left',
                              transform=ax[row][col].transAxes
            )

            test = test + 1
            col = col + 1
            if col == num_cols:
                row = row + 1
                if row == num_rows:
                    return
                col = 0
draw_images()
plt.savefig(os.path.join(DIR_RESULTS, 'images', 'tests-{}.png'.format(actual_date)), bbox_inches='tight')
plt.show()

Guardemos los datos del entrenamiento.

In [ ]:
report_file = 'Resultados - {}.md'.format(actual_date)

with open(os.path.join(DIR_RESULTS, report_file), 'w') as f:
    print('# Resumen de la prueba con fecha {}\n'.format(date_test.strftime('%d/%m/%Y %H:%M:%S')), file=f)

    #  Escribe el tiempo que tardo en realizar el entrenamiento.
    print('Tiempo de entrenamiento del modelo: {}\n.'.format(time.strftime('%H horas %M minutos %S segundos', time.gmtime(training_time))), file=f)

    #  Escribe el resumen de la arquitectura del modelo.
    print('## Arquitectura del modelo.\n\n```', file=f)
    model.summary(print_fn = lambda sum: print(sum + '\n', file=f))
    print('```\n', file=f)

    #  Pon el diagrama de la arquitectura.
    print('![Arquitectura de la red](images/model-{}.png)\n'.format(actual_date), file=f)

    #  Pon las gráficas de pérdida y precisión binaria.
    print('## Pérdida y precisión binaria.\n', file=f)
    print('![Pérdida y precisión binaria](images/loss-binary-accuracy-{}.png)\n'.format(actual_date), file=f)

    #  Pon la matriz de confusión.
    print('## Matriz de confusión.\n', file=f)
    for mat, label in zip(ml_confusion_matrix, labels):
        print('* ' + label + '\n```python', file=f)
        print(mat, file=f)
        print('```\n', file=f)

    #  Escribe el informe con los resultados de la clasificación.
    print('## Resultados de la clasificación.\n\n```', file=f)
    print(cl_report, file=f)
    print('```\n', file=f)

    #  Pon las gráficas de las curvas ROC.
    print('## Curvas ROC de cada etiqueta.\n', file=f)
    print('![Curvas ROC de cada etiqueta](images/ROC-curves-{}.png)\n'.format(actual_date), file=f)


    #  Pon imágenes de las radiografías con los tests.
    print('## Ejemplos de las predicciones del modelo.\n\n', file=f)
    print('    Verde: Clasificación correcta\n', file=f)
    print('    Amarillo: Clasificación parcial\n', file=f)
    print('    Rojo: Clasificación incorrecta\n', file=f)
    print('![Tests](images/tests-{}.png)\n'.format(actual_date), file=f)

Guardamos el modelo creado.

In [ ]:
if SAVE_MODEL:
    model.save(os.path.join(DIR_RESULTS, 'Model-Base-VGG16-Local.{}.h5'.format(actual_date)))